<a href="https://colab.research.google.com/github/nikoardhiansyah/TugasPemrogramanKomputer/blob/main/Kelompok_11_(K)_Tugas_Minggu_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install pandas geopandas rasterio numpy

In [7]:
import gdown
import geopandas as gpd
import pandas as pd
import rasterio

# ----------------------------------------
# 1. Google Drive File URLs & IDs
# ----------------------------------------

# GeoTIFF
tiff_url = "https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0"
# GPKG (survey)
gpkg_url = "https://drive.google.com/uc?id=1AQcHlqzmpQIukLyPIkUnb04-pbjoET_y"
# Perioda CSV
periode_url = "https://drive.google.com/uc?id=1v8_2PwMxl1QGdcbC_RPgUGtYSZsmJDri"

# Output file names
tiff_file = "data_citra.tif"
gpkg_file = "data_survey.gpkg"
periode_file = "perioda.csv"

# ----------------------------------------
# 2. Download Files from Google Drive
# ----------------------------------------

print("Mengunduh data dari Google Drive...")
gdown.download(tiff_url, tiff_file, quiet=False)
gdown.download(gpkg_url, gpkg_file, quiet=False)
gdown.download(periode_url, periode_file, quiet=False)

# ----------------------------------------
# 3. Import Data
# ----------------------------------------

# Import GPKG
gdf = gpd.read_file(gpkg_file).to_crs(epsg=4326)
gdf["lon"] = gdf.geometry.x
gdf["lat"] = gdf.geometry.y
gdf["fid"] = gdf.index.astype(int)

# Rename tanggal → date jika perlu
if "tanggal" in gdf.columns:
    gdf = gdf.rename(columns={"tanggal": "date"})

# Pilih kolom yang diperlukan, pastikan kolom ada
required_cols = ['fid', 'lon', 'lat', 'date', 'fase']
missing_cols = [c for c in required_cols if c not in gdf.columns]
if missing_cols:
    raise ValueError(f"Kolom berikut tidak ditemukan di data survey: {missing_cols}")

output1 = gdf[required_cols].copy()
print("\n✅ Output 1: Filtered survey data")
print(output1.head())

# Import CSV periode
# Specify the separator as ';' because the columns are joined by semicolons
periode_df = pd.read_csv(periode_file, sep=';')
periode_df = periode_df.rename(columns={"Periode": "fase"})
periode_df['fase'] = periode_df['fase'].astype(str).str.strip()
output1['fase'] = output1['fase'].astype(str).str.strip()

# Merge survey dengan periode berdasarkan 'fase'
output2 = pd.merge(output1, periode_df, on="fase", how="left")

# Buat kolom 'periode' dari 'fase' sebagai int untuk band raster
# Ensure that the 'fase' column in output2 only contains values that can be converted to int
try:
    output2["periode"] = output2["fase"].astype(int)
except ValueError as e:
    print(f"Error converting 'fase' to int. Check for non-numeric values: {e}")
    # You might want to inspect the unique values in output2['fase'] here
    print("Unique values in output2['fase']:", output2['fase'].unique())
    raise # Re-raise the error after printing diagnostic info


print("\n✅ Output 2: Data survey + periode")
print(output2.head())

# Buka raster GeoTIFF
raster = rasterio.open(tiff_file)

# Fungsi ambil nilai pixel di koordinat lon, lat pada band tertentu
def get_pixel_value(lon, lat, band):
    try:
        # Check if band is a valid band index (1-based in rasterio)
        if not 1 <= band <= raster.count:
             # print(f"Warning: Band index {band} is out of valid range (1 to {raster.count}).")
             return None # Or handle as appropriate for your data

        # Sample the raster. rasterio.sample.sample_gen expects a list of points.
        # It returns a generator, so we iterate to get the value.
        for val in rasterio.sample.sample_gen(raster, [(lon, lat)], indexes=[band]):
            return val[0] # sample_gen returns an array of values, one per requested band
    except Exception as e:
        print(f"Error ambil pixel di ({lon}, {lat}) band {band}: {e}")
        return None

# Ekstrak nilai pixel citra sesuai periode (band)
# Apply the function to each row. The 'periode' column should now contain integer band indices.
output2["p0"] = output2.apply(lambda row: get_pixel_value(row["lon"], row["lat"], row["periode"]), axis=1)

# Output 3 dengan kolom lengkap
output3 = output2[['fid', 'lon', 'lat', 'date', 'fase', 'periode', 'p0']]
print("\n✅ Output 3: Data lengkap dengan nilai pixel")
print(output3.head())

# Simpan hasil jika ingin
output3.to_csv("output3_with_pixel.csv", index=False)
print("\n✅ Hasil disimpan di output3_with_pixel.csv")


Mengunduh data dari Google Drive...


Downloading...
From (original): https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0
From (redirected): https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0&confirm=t&uuid=757ea7f8-19a0-4078-9f6c-31fba28b2c1a
To: /content/data_citra.tif

  0%|          | 0.00/1.92G [00:00<?, ?B/s]
  0%|          | 2.10M/1.92G [00:00<01:40, 19.1MB/s]
  1%|          | 11.0M/1.92G [00:00<00:42, 45.1MB/s]
  1%|          | 19.4M/1.92G [00:00<00:49, 38.3MB/s]
  1%|▏         | 25.7M/1.92G [00:00<00:43, 44.0MB/s]
  2%|▏         | 32.5M/1.92G [00:00<00:37, 49.8MB/s]
  2%|▏         | 39.8M/1.92G [00:00<00:33, 56.2MB/s]
  2%|▏         | 46.1M/1.92G [00:01<00:48, 38.4MB/s]
  3%|▎         | 58.7M/1.92G [00:01<00:46, 39.9MB/s]
  4%|▍         | 73.4M/1.92G [00:01<00:31, 58.3MB/s]
  4%|▍         | 81.3M/1.92G [00:01<00:38, 47.7MB/s]
  5%|▍         | 91.2M/1.92G [00:01<00:36, 49.8MB/s]
  5%|▌         | 102M/1.92G [00:02<00:30, 60.6MB/s] 
  6%|▌         | 110M/1.92G [00:02<00:36, 49.4MB/s]
  6%|▋


✅ Output 1: Filtered survey data
   fid         lon       lat       date  fase
0    0  107.463959 -6.260439  3/18/2024     1
1    1  107.464081 -6.260284  3/18/2024     1
2    2  107.463608 -6.260762  3/18/2024     1
3    3  107.461742 -6.261511  3/18/2024     1
4    4  107.463209 -6.260423  3/18/2024     1

✅ Output 2: Data survey + periode
   fid         lon       lat       date fase  Start Date    End Date  periode
0    0  107.463959 -6.260439  3/18/2024    1  2024-01-01  2024-01-13        1
1    1  107.464081 -6.260284  3/18/2024    1  2024-01-01  2024-01-13        1
2    2  107.463608 -6.260762  3/18/2024    1  2024-01-01  2024-01-13        1
3    3  107.461742 -6.261511  3/18/2024    1  2024-01-01  2024-01-13        1
4    4  107.463209 -6.260423  3/18/2024    1  2024-01-01  2024-01-13        1

✅ Output 3: Data lengkap dengan nilai pixel
   fid         lon       lat       date fase  periode    p0
0    0  107.463959 -6.260439  3/18/2024    1        1 -1379
1    1  107.464081 -6.